In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq
llm = ChatGroq(model = "gemma2-9b-it", temperature = 0)

In [4]:
def multiply(a:int, b:int) -> int:
    '''int a and int b are the two numbers which needs to be multiplied'''
    return a * b

def add(a:int, b:int) -> int:
    '''int a and int b are the two numbers which needs to be added'''
    return a + b

def div(a:int, b:int) -> float:
    '''int a and int b are the two numbers which needs to be divided'''
    return a * b

def sub(a:int, b:int) -> int:
    '''int a and int b are the two numbers which needs to be subtracted'''
    return a * b



In [5]:
tools = [add, multiply, sub, div]

In [8]:
llm_with_tools = llm.bind_tools(tools)
llm_with_tools

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000019A75CF4B60>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000019A75E50230>, model_name='gemma2-9b-it', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'add', 'description': 'int a and int b are the two numbers which needs to be added', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'multiply', 'description': 'int a and int b are the two numbers which needs to be multiplied', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'sub', 'description': 'int a and int b are the two numbers which needs to be subtracted', 'parameters': {'prop

In [10]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState
from langgraph.prebuilt import tools_condition, ToolNode
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [38]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{input}")
])

In [39]:
def assistant(state: dict):
    # take the last human message from state
    user_input = state["messages"][-1].content  

    # format the prompt correctly
    formatted = prompt.format_prompt(input=user_input)

    # call the LLM with a PromptValue (✅ valid type)
    response = llm.invoke(formatted)

    return {"messages": [response]}

In [40]:
builder = StateGraph(MessagesState)

builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition,)
builder.add_edge("tools", "assistant")

memory = MemorySaver()

graph = builder.compile(checkpointer=memory, interrupt_before=["assistant"])

In [41]:
thread = {"configurable":{"thread_id":"1"}}

In [42]:
input = {"messages":HumanMessage(content = "Multiply 2 and 3")}

In [43]:
for event in graph.stream(input, thread, stream_mode = "values"):
    event["messages"][-1].pretty_print()


================================ Human Message =================================

Multiply 2 and 3


In [44]:
for event in graph.stream(None, thread, stream_mode = "values"):
    event["messages"][-1].pretty_print()


================================ Human Message =================================

Multiply 2 and 3
================================== Ai Message ==================================

2 multiplied by 3 is 6.


In [45]:
for event in graph.stream(None, thread, stream_mode = "values"):
    event["messages"][-1].pretty_print()


================================== Ai Message ==================================

2 multiplied by 3 is 6.
